In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import lightgbm as lgb
import joblib

from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score,
    balanced_accuracy_score,
    f1_score,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.utils import shuffle


# ============================================================
# 0) Paths and basic settings
# ============================================================

DATA_PATH = Path(r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_ALL_FINAL_FROM_DATA_ALL.csv")
MODELS_DIR = Path(r"C:\Users\LENOVO\Desktop\sen\sen\models_nested_lgbm_nlnso_hybrid_full")

os.makedirs(MODELS_DIR, exist_ok=True)

SUBJECT_COL = "subject"
LABEL_COL   = "label"

RANDOM_STATE   = 42
N_OUTER_FOLDS  = 10
N_INNER_FOLDS  = 10  # still 10 inner folds; only hyperparameter combos reduced to 3

np.random.seed(RANDOM_STATE)


# ============================================================
# 1) Metadata columns (NOT used as features)
# ============================================================

meta_cols = [
    "subject",
    "run",
    "window_idx",
    "label",
    "run_base",
    "ecg_start_time_sec",
    "t_start",
    "t_end",
    "win_idx",
    "Unnamed: 0",
    "timestamp_center",
]


# ============================================================
# 2) Helper functions
# ============================================================

def make_subject_folds(subject_ids, n_folds=10, random_state=42):
    """
    Split unique subject IDs into n_folds at subject level.
    Returns a list of arrays, each array = subject IDs in that fold.
    """
    unique_subjects = np.array(sorted(np.unique(subject_ids)))
    rng = np.random.RandomState(random_state)
    rng.shuffle(unique_subjects)
    folds = np.array_split(unique_subjects, n_folds)
    return folds


def hybrid_resample_l012(
    X,
    y,
    majority_class=0,
    mid_class=1,
    minority_class=2,
    max_majority_ratio=2.0,
    random_state=42,
):
    """
    Hybrid sampling strategy for labels {0, 1, 2}:

    - Class 0 (majority, interictal): undersampling.
    - Class 1 (preictal): mild oversampling (~1.5x).
    - Class 2 (seizure): oversampling to ~0.7 * class1,
      but limited to ≤ 10 × original seizure count.

    This MUST be applied ONLY on training data (never on test).
    """
    rng = np.random.RandomState(random_state)
    X = np.asarray(X)
    y = np.asarray(y)

    idx_maj = np.where(y == majority_class)[0]
    idx_mid = np.where(y == mid_class)[0]
    idx_min = np.where(y == minority_class)[0]

    n_maj = len(idx_maj)
    n_mid = len(idx_mid)
    n_min = len(idx_min)

    if n_mid == 0 or n_min == 0:
        raise ValueError("Classes 1 and 2 must be present for hybrid_resample_l012.")

    # 1) Target sizes for each class
    mid_oversample_factor = 1.5
    target_mid = max(n_mid, int(n_mid * mid_oversample_factor))

    minority_target_ratio = 0.7
    raw_target_min = int(target_mid * minority_target_ratio)
    max_minority_factor = 10.0
    max_min = int(n_min * max_minority_factor)
    target_min = max(n_min, min(raw_target_min, max_min))

    target_maj = min(n_maj, int(max_majority_ratio * target_mid))

    # 2) Sample indices per class

    # Majority class (0) – undersampling
    if n_maj > target_maj:
        idx_maj_keep = rng.choice(idx_maj, size=target_maj, replace=False)
    else:
        idx_maj_keep = idx_maj

    # Mid class (1) – mild oversampling to reach target_mid
    if n_mid >= target_mid:
        idx_mid_keep = rng.choice(idx_mid, size=target_mid, replace=False)
    else:
        extra = target_mid - n_mid
        extra_idx = rng.choice(idx_mid, size=extra, replace=True)
        idx_mid_keep = np.concatenate([idx_mid, extra_idx])

    # Minority class (2) – oversampling with safety cap
    if n_min >= target_min:
        idx_min_keep = rng.choice(idx_min, size=target_min, replace=False)
    else:
        extra = target_min - n_min
        extra_idx = rng.choice(idx_min, size=extra, replace=True)
        idx_min_keep = np.concatenate([idx_min, extra_idx])

    # 3) Merge class indices and shuffle
    all_idx = np.concatenate([idx_maj_keep, idx_mid_keep, idx_min_keep])
    all_idx = shuffle(all_idx, random_state=random_state)

    X_res = X[all_idx]
    y_res = y[all_idx]

    return X_res, y_res


def build_lgbm(params, num_classes):
    """
    Create a LightGBM classifier with:
      - multiclass objective
      - explicit class weights (0,1,2)
      - GPU enabled (device_type="gpu")

    NOTE:
    This requires a LightGBM build compiled with GPU support.
    """
    # Class weights to compensate class imbalance
    class_weight = {0: 1.0, 1: 2.0, 2: 3.0}

    return lgb.LGBMClassifier(
        objective="multiclass",
        num_class=num_classes,
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        num_leaves=params["num_leaves"],
        learning_rate=params["learning_rate"],
        subsample=params["subsample"],
        colsample_bytree=params["colsample_bytree"],
        class_weight=class_weight,  # <-- class weights applied here
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=-1,
        # ====== GPU settings ======
        device_type="gpu",      # Force GPU usage
        tree_learner="data",    # Good default for GPU
        gpu_platform_id=0,
        gpu_device_id=0,
        max_bin=255,            # Default, GPU-friendly
    )


def lgbm_fit_predict_proba(X_train, y_train, X_val, params, num_classes):
    """
    Train LightGBM on a training subset and return class probability
    predictions on the validation subset (used in inner-loop ROC AUC).
    """
    model = build_lgbm(params, num_classes)
    model.fit(X_train, y_train)
    y_val_proba = model.predict_proba(X_val)
    return model, y_val_proba


def lgbm_fit_predict(X_train, y_train, X_test, params, num_classes):
    """
    Train LightGBM on the (resampled) training data and return
    hard-label predictions for the test set (used in the outer loop).
    """
    model = build_lgbm(params, num_classes)
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    return model, y_test_pred


def evaluate_metrics(y_true, y_pred, label_set=None):
    """
    Compute per-class and global metrics, plus confusion matrix
    and TP/FP/FN/TN per class.
    """
    if label_set is None:
        label_set = np.unique(y_true)

    prec, rec, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=label_set, zero_division=0
    )

    cm = confusion_matrix(y_true, y_pred, labels=label_set)
    total = cm.sum()

    tp = []
    fp = []
    fn = []
    tn = []

    for i in range(len(label_set)):
        TP = cm[i, i]
        FP = cm[:, i].sum() - TP
        FN = cm[i, :].sum() - TP
        TN = total - (TP + FP + FN)
        tp.append(TP)
        fp.append(FP)
        fn.append(FN)
        tn.append(TN)

    metrics = {
        "labels": label_set,
        "precision_per_class": prec,
        "recall_per_class": rec,
        "f1_per_class": f1,
        "support_per_class": support,
        "tp_per_class": np.array(tp),
        "fp_per_class": np.array(fp),
        "fn_per_class": np.array(fn),
        "tn_per_class": np.array(tn),
        "accuracy": accuracy_score(y_true, y_pred),
        "balanced_accuracy": balanced_accuracy_score(y_true, y_pred),
        "micro_f1": f1_score(y_true, y_pred, average="micro"),
        "macro_f1": f1_score(y_true, y_pred, average="macro"),
        "weighted_f1": f1_score(y_true, y_pred, average="weighted"),
        "confusion_matrix": cm,
    }

    return metrics


def print_metrics(metrics, header=""):
    """
    Pretty-print per-class and global metrics + TP/FP/FN/TN per label.
    """
    labels  = metrics["labels"]
    prec    = metrics["precision_per_class"]
    rec     = metrics["recall_per_class"]
    f1      = metrics["f1_per_class"]
    support = metrics["support_per_class"]
    cm      = metrics["confusion_matrix"]

    if header:
        print("\n" + "=" * 70)
        print(header)
        print("=" * 70)

    print("\nPer-class metrics:")
    print("label\tprecision\trecall\t\tf1-score\tsupport")
    for i, c in enumerate(labels):
        print(f"{c}\t{prec[i]:.4f}\t\t{rec[i]:.4f}\t\t{f1[i]:.4f}\t\t{support[i]}")

    print("\nGlobal metrics:")
    print(f"Accuracy          : {metrics['accuracy']:.4f}")
    print(f"Balanced Accuracy : {metrics['balanced_accuracy']:.4f}")
    print(f"Micro-F1          : {metrics['micro_f1']:.4f}")
    print(f"Macro-F1          : {metrics['macro_f1']:.4f}")
    print(f"Weighted-F1       : {metrics['weighted_f1']:.4f}")

    print("\nConfusion matrix (rows = true, cols = predicted):")
    print(cm)

    total = cm.sum()
    print("\nPer-class confusion details (TP, FP, FN, TN):")
    for idx, c in enumerate(labels):
        TP = cm[idx, idx]
        FP = cm[:, idx].sum() - TP
        FN = cm[idx, :].sum() - TP
        TN = total - (TP + FP + FN)
        print(f"Label {c}: TP={TP}, FP={FP}, FN={FN}, TN={TN}")


def compute_inner_roc_auc_for_params(
    X, y, subjects, label_set, param_grid, n_inner_folds=10
):
    """
    Inner loop for nested CV:

    - Split Train+Val subjects into n_inner_folds (subject-wise).
    - For each hyperparameter combination:
        - Train on inner-train subjects (with hybrid sampling).
        - Validate on inner-val subjects.
        - Compute macro ROC AUC (multi-class 'ovr').

    Returns:
        best_params: dict of best hyperparameters.
        best_auc:   mean macro ROC AUC across inner folds.
    """
    inner_folds = make_subject_folds(
        subjects, n_folds=n_inner_folds, random_state=RANDOM_STATE
    )

    combo_scores = []

    print("\n------------------------------------------------------")
    print("Inner loop ROC AUC hyperparameter tuning (LightGBM, GPU)")
    print("------------------------------------------------------")

    for combo_idx, params in enumerate(param_grid):
        fold_aucs = []

        for inner_idx, inner_val_subjects in enumerate(inner_folds):
            is_inner_val   = np.isin(subjects, inner_val_subjects)
            is_inner_train = ~is_inner_val

            X_inner_train = X[is_inner_train]
            y_inner_train = y[is_inner_train]
            X_inner_val   = X[is_inner_val]
            y_inner_val   = y[is_inner_val]

            # Hybrid resampling applied only on inner-train
            X_inner_train_bal, y_inner_train_bal = hybrid_resample_l012(
                X_inner_train,
                y_inner_train,
                majority_class=0,
                mid_class=1,
                minority_class=2,
                max_majority_ratio=2.0,
                random_state=RANDOM_STATE + inner_idx,
            )

            # Fit on balanced train and get probabilities on validation set
            _, y_inner_val_proba = lgbm_fit_predict_proba(
                X_inner_train_bal,
                y_inner_train_bal,
                X_inner_val,
                params,
                num_classes=len(label_set),
            )

            try:
                auc = roc_auc_score(
                    y_inner_val,
                    y_inner_val_proba,
                    labels=label_set,
                    multi_class="ovr",
                    average="macro",
                )
                fold_aucs.append(auc)
            except ValueError:
                # Can happen if a class is missing in y_inner_val
                continue

            # end inner fold loop

        mean_auc = float(np.mean(fold_aucs)) if len(fold_aucs) > 0 else np.nan
        combo_scores.append((params, mean_auc))
        print(f"Combo {combo_idx + 1}/{len(param_grid)}: "
              f"{params} --> mean macro ROC AUC = {mean_auc:.4f}")

    combo_scores = [c for c in combo_scores if not np.isnan(c[1])]
    if len(combo_scores) == 0:
        raise RuntimeError("All inner ROC AUC scores are NaN. Check your data splits or labels.")

    best_params, best_auc = max(combo_scores, key=lambda t: t[1])

    print("\nBest inner hyperparameters based on ROC AUC (LightGBM, GPU):")
    print(best_params)
    print(f"Best inner mean macro ROC AUC: {best_auc:.4f}")

    return best_params, best_auc


# ============================================================
# 3) Load data
# ============================================================

print("Loading fused dataset...")
df = pd.read_csv(DATA_PATH)

if SUBJECT_COL not in df.columns or LABEL_COL not in df.columns:
    raise ValueError("Check SUBJECT_COL and LABEL_COL names. They are not found in the dataframe.")

print(f"Data shape: {df.shape}")
print("\nLabel distribution:")
print(df[LABEL_COL].value_counts())
print("\nLabel proportions (%):")
print(df[LABEL_COL].value_counts(normalize=True) * 100)

print("\nMetadata columns dropped from training:")
print(meta_cols)

feature_cols = [c for c in df.columns if c not in meta_cols]
print(f"\nNumber of feature columns used for training: {len(feature_cols)}")

X_all = df[feature_cols].astype("float32").values
y_all = df[LABEL_COL].values
subjects_all = df[SUBJECT_COL].values

label_set = np.unique(y_all)
print(f"Unique labels: {label_set}")


# ============================================================
# 4) Hyperparameter grid for LightGBM (inner ROC AUC tuning)
#    Reduced to 3 combinations instead of 24
# ============================================================

param_grid = [
    {
        "n_estimators": 300,
        "max_depth": 12,
        "num_leaves": 40,
        "learning_rate": 0.1,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
    },
    {
        "n_estimators": 400,
        "max_depth": 18,
        "num_leaves": 60,
        "learning_rate": 0.05,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
    },
    {
        "n_estimators": 500,
        "max_depth": 12,
        "num_leaves": 60,
        "learning_rate": 0.1,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
    },
]

print(f"\nTotal hyperparameter combinations in inner loop: {len(param_grid)}")


# ============================================================
# 5) Nested 10x10 N-LNSO with ROC AUC inner tuning (LightGBM, GPU)
# ============================================================

outer_folds = make_subject_folds(
    subjects_all,
    n_folds=N_OUTER_FOLDS,
    random_state=RANDOM_STATE
)

all_test_y_true = []
all_test_y_pred = []

outer_best_params_list = []
outer_macro_f1_list = []
outer_best_auc_list = []

for outer_idx, outer_test_subjects in enumerate(outer_folds):
    print("\n" + "#" * 80)
    print(f"OUTER Fold {outer_idx + 1}/{N_OUTER_FOLDS}")
    print("#" * 80)

    is_test = np.isin(subjects_all, outer_test_subjects)
    is_trainval = ~is_test

    X_trainval = X_all[is_trainval]
    y_trainval = y_all[is_trainval]
    subjects_trainval = subjects_all[is_trainval]

    X_test = X_all[is_test]
    y_test = y_all[is_test]
    subjects_test = subjects_all[is_test]

    print(f"Train+Val subjects: {len(np.unique(subjects_trainval))}, "
          f"Test subjects: {len(np.unique(subjects_test))}")

    # Inner loop: ROC AUC-based hyperparameter tuning
    best_inner_params, best_inner_auc = compute_inner_roc_auc_for_params(
        X_trainval, y_trainval, subjects_trainval,
        label_set=label_set,
        param_grid=param_grid,
        n_inner_folds=N_INNER_FOLDS
    )

    outer_best_params_list.append(best_inner_params)
    outer_best_auc_list.append(best_inner_auc)

    # Train final model on all Train+Val with hybrid resampling
    X_trainval_bal, y_trainval_bal = hybrid_resample_l012(
        X_trainval, y_trainval,
        majority_class=0,
        mid_class=1,
        minority_class=2,
        max_majority_ratio=2.0,
        random_state=RANDOM_STATE + outer_idx
    )

    final_model, y_test_pred = lgbm_fit_predict(
        X_trainval_bal, y_trainval_bal,
        X_test,
        best_inner_params,
        num_classes=len(label_set)
    )

    model_path = MODELS_DIR / f"lgbm_nested_outer_{outer_idx + 1:02d}.pkl"
    joblib.dump(final_model, model_path)
    print(f"Saved outer fold model to: {model_path}")

    fold_metrics = evaluate_metrics(y_test, y_test_pred, label_set)
    print_metrics(fold_metrics, header=f"OUTER Fold {outer_idx + 1} Test Metrics")

    all_test_y_true.append(y_test)
    all_test_y_pred.append(y_test_pred)
    outer_macro_f1_list.append(fold_metrics["macro_f1"])


# ============================================================
# 6) Global metrics across all outer folds
# ============================================================

all_test_y_true = np.concatenate(all_test_y_true)
all_test_y_pred = np.concatenate(all_test_y_pred)

global_metrics = evaluate_metrics(all_test_y_true, all_test_y_pred, label_set)
print_metrics(global_metrics, header="GLOBAL Test Metrics Across All OUTER Folds (LightGBM, GPU)")

print("\nNested 10×10 N-LNSO evaluation completed.")
print("All outer-fold models are saved and ready for external testing.")


# ============================================================
# 7) Summary of best inner-loop ROC AUC per outer fold + CSV
# ============================================================

print("\n" + "=" * 70)
print("Summary of inner-loop ROC AUC used to select best hyperparameters")
print("=" * 70)

auc_rows = []
for i, (p, auc_val) in enumerate(zip(outer_best_params_list, outer_best_auc_list), start=1):
    print(f"OUTER Fold {i:02d}: best inner mean macro ROC AUC = {auc_val:.4f}")
    print(f"  Selected params: {p}")
    row = {
        "outer_fold": i,
        "best_inner_mean_macro_roc_auc": auc_val,
        "n_estimators": p["n_estimators"],
        "max_depth": p["max_depth"],
        "num_leaves": p["num_leaves"],
        "learning_rate": p["learning_rate"],
        "subsample": p["subsample"],
        "colsample_bytree": p["colsample_bytree"],
    }
    auc_rows.append(row)

inner_auc_df = pd.DataFrame(auc_rows)
inner_auc_csv_path = MODELS_DIR / "inner_auc_summary_lgbm_hybrid_full_gpu.csv"
inner_auc_df.to_csv(inner_auc_csv_path, index=False)

print("=" * 70)
print(f"Inner-loop ROC AUC summary saved to: {inner_auc_csv_path}")
print("=" * 70)


# ============================================================
# 8) Train a single FINAL deployment model on all subjects
#    using the best hyperparameters discovered in nested CV
# ============================================================

from collections import defaultdict

combo_counts = defaultdict(int)
combo_f1_sum = defaultdict(float)

for params, macro_f1 in zip(outer_best_params_list, outer_macro_f1_list):
    key = tuple(sorted(params.items()))
    combo_counts[key] += 1
    combo_f1_sum[key] += float(macro_f1)

best_key = None
best_count = -1
best_f1_sum = -np.inf

for key in combo_counts:
    count = combo_counts[key]
    f1_sum = combo_f1_sum[key]
    if (count > best_count) or (count == best_count and f1_sum > best_f1_sum):
        best_count = count
        best_f1_sum = f1_sum
        best_key = key

final_params = dict(best_key)

print("\n" + "=" * 70)
print("Selected FINAL deployment hyperparameters (based on outer folds, GPU):")
print(final_params)
print(f"Selected combo frequency across outer folds: {best_count}")
print("=" * 70)

# Apply hybrid sampling on the full dataset (all subjects)
X_full_bal, y_full_bal = hybrid_resample_l012(
    X_all, y_all,
    majority_class=0,
    mid_class=1,
    minority_class=2,
    max_majority_ratio=2.0,
    random_state=RANDOM_STATE + 999
)

final_deployment_model = build_lgbm(final_params, num_classes=len(label_set))
final_deployment_model.fit(X_full_bal, y_full_bal)

deployment_model_path = MODELS_DIR / "lgbm_final_deployment_hybrid_full_gpu.pkl"
joblib.dump(final_deployment_model, deployment_model_path)

print("\nFinal LightGBM deployment model (GPU) trained on all subjects and saved to:")
print(deployment_model_path)
print("Use this model for real-time or external testing on new patients.")

Loading fused dataset...
Data shape: (1622677, 86)

Label distribution:
label
0    1312916
1     289844
2      19917
Name: count, dtype: int64

Label proportions (%):
label
0    80.910495
1    17.862088
2     1.227416
Name: proportion, dtype: float64

Metadata columns dropped from training:
['subject', 'run', 'window_idx', 'label', 'run_base', 'ecg_start_time_sec', 't_start', 't_end', 'win_idx', 'Unnamed: 0', 'timestamp_center']

Number of feature columns used for training: 75
Unique labels: [0 1 2]

Total hyperparameter combinations in inner loop: 3

################################################################################
OUTER Fold 1/10
################################################################################
Train+Val subjects: 101, Test subjects: 12

------------------------------------------------------
Inner loop ROC AUC hyperparameter tuning (LightGBM, GPU)
------------------------------------------------------


c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LG

Combo 1/3: {'n_estimators': 300, 'max_depth': 12, 'num_leaves': 40, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 0.8} --> mean macro ROC AUC = 0.6058


c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LG

Combo 2/3: {'n_estimators': 400, 'max_depth': 18, 'num_leaves': 60, 'learning_rate': 0.05, 'subsample': 0.8, 'colsample_bytree': 0.8} --> mean macro ROC AUC = 0.6069


...

Combo 2/3: {'n_estimators': 400, 'max_depth': 18, 'num_leaves': 60, 'learning_rate': 0.05, 'subsample': 0.8, 'colsample_bytree': 0.8} --> mean macro ROC AUC = 0.5934


c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LG

Combo 3/3: {'n_estimators': 500, 'max_depth': 12, 'num_leaves': 60, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 0.8} --> mean macro ROC AUC = 0.5898

Best inner hyperparameters based on ROC AUC (LightGBM, GPU):
{'n_estimators': 400, 'max_depth': 18, 'num_leaves': 60, 'learning_rate': 0.05, 'subsample': 0.8, 'colsample_bytree': 0.8}
Best inner mean macro ROC AUC: 0.5934


c:\Users\LENOVO\Desktop\sen\sen\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Saved outer fold model to: C:\Users\LENOVO\Desktop\sen\sen\models_nested_lgbm_nlnso_hybrid_full\lgbm_nested_outer_10.pkl

OUTER Fold 10 Test Metrics

Per-class metrics:
label	precision	recall		f1-score	support
0	0.8417		0.7442		0.7900		146289
1	0.2284		0.2749		0.2495		29531
2	0.0712		0.5471		0.1259		1636

Global metrics:
Accuracy          : 0.6643
Balanced Accuracy : 0.5220
Micro-F1          : 0.6643
Macro-F1          : 0.3885
Weighted-F1       : 0.6939

Confusion matrix (rows = true, cols = predicted):
[[108868  27233  10188]
 [ 19919   8117   1495]
 [   549    192    895]]

Per-class confusion details (TP, FP, FN, TN):
Label 0: TP=108868, FP=20468, FN=37421, TN=10699
Label 1: TP=8117, FP=27425, FN=21414, TN=120500
Label 2: TP=895, FP=11683, FN=741, TN=164137

GLOBAL Test Metrics Across All OUTER Folds (LightGBM, GPU)

Per-class metrics:
label	precision	recall		f1-score	support
0	0.8115		0.6918		0.7469		1312916
1	0.1867		0.2050		0.1954		289844
2	0.0548		0.5104		0.0990		19917

Global m